In [260]:
import pandas as pd
import inflection as inf

# Quantifying Monster Actions
In this notebook, actions taken by a monster will be quantified. This will mostly focus on attack actions. Monsters typically look something like the following:

![title](img/Aboleth_actions.png)

The descriptions of each action was scraped and stored in `actions.csv`.

In [261]:
df = pd.read_csv('actions.csv')
df.head(10)

,Monster Name,Name,Description,Type
0,Aboleth,Amphibious.,The aboleth can breathe air and water.,extra
1,Aboleth,Mucous Cloud.,"While underwater, the aboleth is surrounded by...",extra
2,Aboleth,Probing Telepathy.,If a creature communicates telepathically with...,extra
3,Aboleth,Multiattack.,The aboleth makes three tentacle attacks.,action
4,Aboleth,Tentacle.,"Melee Weapon Attack: +9 to hit, reach 10 ft., ...",action
5,Aboleth,Tail.,"Melee Weapon Attack: +9 to hit, reach 10 ft. o...",action
6,Aboleth,Enslave .,The aboleth targets one creature it can see wi...,action
7,Aboleth,Detect.,The aboleth makes a Wisdom (Perception) check.,legend
8,Aboleth,Tail Swipe.,The aboleth makes one tail attack.,legend
9,Aboleth,Psychic Drain .,One creature charmed by the aboleth takes 10 (...,legend


## Understanding how Actions contribute to attack
The part of the monster that has a rigid application in the game mechanics is the "actions" section. This is where each monsters attacks are described. There are also "extras" that tend to contain special information about about what a monster can and can't do (jump high, breathe water, etc.). Some monsters have "Legendary Actions" that can be played at the end of each characters turn, but only once per monster turn. For now, I will focus on just 'actions'. 

In [262]:
df_actions = df[df.Type == 'action']
df_actions.head(10)

,Monster Name,Name,Description,Type
3,Aboleth,Multiattack.,The aboleth makes three tentacle attacks.,action
4,Aboleth,Tentacle.,"Melee Weapon Attack: +9 to hit, reach 10 ft., ...",action
5,Aboleth,Tail.,"Melee Weapon Attack: +9 to hit, reach 10 ft. o...",action
6,Aboleth,Enslave .,The aboleth targets one creature it can see wi...,action
13,Deva,Multiattack.,The deva makes two melee attacks.,action
14,Deva,Mace.,"Melee Weapon Attack: +8 to hit, reach 5 ft., o...",action
15,Deva,Healing Touch .,The deva touches another creature. The target ...,action
16,Deva,Change Shape.,The deva magically polymorphs into a humanoid ...,action
21,Planetar,Multiattack.,The planetar makes two melee attacks.,action
22,Planetar,Greatsword.,"Melee Weapon Attack: +12 to hit, reach 5 ft., ...",action


I don't like the '.' in the name of each attack, and I do not trust that the spacing is consistant. Let's strip it. Let's also strip any potential hidden white space. Then, let's look at all of the unique action names that there are.

In [263]:
df_actions.Name = df_actions.Name.apply(lambda x : x.strip('. '))

In [264]:
df_actions.head()

,Monster Name,Name,Description,Type
3,Aboleth,Multiattack,The aboleth makes three tentacle attacks.,action
4,Aboleth,Tentacle,"Melee Weapon Attack: +9 to hit, reach 10 ft., ...",action
5,Aboleth,Tail,"Melee Weapon Attack: +9 to hit, reach 10 ft. o...",action
6,Aboleth,Enslave,The aboleth targets one creature it can see wi...,action
13,Deva,Multiattack,The deva makes two melee attacks.,action


I will need to be searching for substrings within the attack description at some point. Let's make everything lower case to make this easier.

In [265]:
df_actions[['Name','Description']] = df_actions[['Name','Description']].apply(lambda x : x.str.lower())

In [266]:
df_actions.groupby("Name").Type.count().head(20)

Name
acid breath           7
acid spray            1
animate chains        1
animate trees         1
antennae              1
battleaxe             1
beak                 12
beaks                 1
beard                 1
bite                138
bites                 6
blinding breath       1
blinding spittle      1
blood drain           1
breath weapons       20
chain                 1
change shape         11
charm                 2
claw                 54
claws                29
Name: Type, dtype: int64

It looks like there are some plural versions of actions. This is worth investigating. Let's take a look at the difference between 'Claw' and 'Claws'.

In [267]:
df_claw = df_actions[df_actions.Name == 'claw']
print(df_claw.iloc[1].Description)
df_claw.head()

melee weapon attack: +10 to hit, reach 10 ft., one target. hit: 15 (3d6 + 5) slashing damage.


,Monster Name,Name,Description,Type
122,Hezrou,claw,"melee weapon attack: +7 to hit, reach 5 ft., o...",action
133,Nalfeshnee,claw,"melee weapon attack: +10 to hit, reach 10 ft.,...",action
150,Barbed Devil,claw,"melee weapon attack: +6 to hit, reach 5 ft., o...",action
162,Bone Devil,claw,"melee weapon attack: +8 to hit, reach 10 ft., ...",action
200,Pit Fiend,claw,"melee weapon attack: +14 to hit, reach 10 ft.,...",action


In [268]:
df_claws = df_actions[df_actions.Name == 'claws']
print(df_claws.iloc[1].Description)
df_claws.head()

melee weapon attack: +2 to hit, reach 5 ft., one target. hit: 5 (2d4) slashing damage.


,Monster Name,Name,Description,Type
75,Chimera,claws,"melee weapon attack: +7 to hit, reach 5 ft., o...",action
111,Dretch,claws,"melee weapon attack: +2 to hit, reach 5 ft., o...",action
138,Quasit,claws,"melee weapon attack: +4 to hit, reach 5 ft., o...",action
184,Ice Devil,claws,"melee weapon attack: +10 to hit, reach 5 ft., ...",action
595,Ettercap,claws,"melee weapon attack: +4 to hit, reach 5 ft., o...",action


At first glance, it appears as if the difference between plural attacks and singular attacks is syntactic and has no bearing on the way the attack is staged. We can double check this by searching each attack for the occurance of a digit. The singular digits should suffice in confirming this hypothesis.

In [269]:
number_word_list = ['two','three','four','five','six','seven','eight','nine'] 
#Made a list in case I want to use it later
number_word_string = ''
for number in number_word_list:
    number_word_string += number + "|"
number_word_string = number_word_string.strip('|')

df_multiple = df_actions[df_actions.Description.str.contains(number_word_string)]
df_multiple.groupby("Name").Type.count().head(20)

Name
animate chains       1
animate trees        1
claw                 2
create specter       1
ethereal stride      1
fey charm            1
lightning storm      1
longsword            7
multiattack        136
phantasms            1
pincer               2
quarterstaff         1
spear                8
teleport             1
tentacle             1
warhammer            1
whelm                1
Name: Type, dtype: int64

### Understanding Multiattack
`Multiattack` has by far the most entries. Let's examin a set of multiattacks to understand what it does.

In [270]:
df_multiple[df_multiple.Name == 'multiattack'].Description.head(10)

3             the aboleth makes three tentacle attacks.
13                    the deva makes two melee attacks.
21                the planetar makes two melee attacks.
28              the solar makes two greatsword attacks.
38                   the armor makes two melee attacks.
55    the behir makes two attacks: one with its bite...
68    the centaur makes two attacks: one with its pi...
72    the chimera makes three attacks: one with its ...
79    the chuul makes two pincer attacks. if the chu...
85    the cloaker makes two attacks: one with its bi...
Name: Description, dtype: object

It looks as though `multiattack` just defines which attacks a monster can do if they can make more than one attack. To confirm this, let's do a search on `makes n attacks` and make sure the list is the same size.

In [271]:
print(len(df_multiple[(df_multiple.Name == 'multiattack') & df_multiple.Description.str.contains('makes ' + number_word_string + " attacks" ,regex=True)].Description))
print(len(df_multiple[df_multiple.Name == 'multiattack'].Description))

136
136


They are the same! Great! We understand what multiattack does. We should remember this for later.

### The other actions with numbers in them

Let's look at the remaining actions with english numbers > 1, but specifically if it references to an attack and includes damage or hitting.

In [272]:
df_multiple_no_ma = df_actions[df_actions.Description.str.contains(number_word_string) & ~(df_actions.Name == 'multiattack') & (df_actions.Description.str.contains('damage|hit') & (df_actions.Description.str.contains('attack'))) ]
print(len(df_multiple_no_ma))
print(df_multiple_no_ma.iloc[11].Description)

24
melee or ranged weapon attack: +4 to hit, reach 5 ft. or range 20/60 ft., one creature. hit: 5 (1d6 + 2) piercing damage, or 6 (1d8 + 2) piercing damage if used with two hands to make a melee attack.


Many of the remaining actions look like they all attack one creature or one target, but have logic depending on the number of hands being used.

In [273]:
df_multiple_no_ma_no_hands = df_multiple_no_ma[~df_multiple_no_ma.Description.str.contains('two hands')]
for i in range(len(df_multiple_no_ma_no_hands)):
    print(df_multiple_no_ma_no_hands.iloc[i].Description + '\n')

melee weapon attack: +6 to hit, reach 10 ft., one target. hit: 11 (2d6 + 4) bludgeoning damage. the target is grappled (escape dc 14) if it is a large or smaller creature and the chuul doesn't have two other creatures grappled.

the cloaker magically creates three illusory duplicates of itself if it isn't in bright light. the duplicates move with it and mimic its actions, shifting position so as to make it impossible to track which cloaker is the real one. if the cloaker is ever in an area of bright light, the duplicates disappear.
whenever any creature targets the cloaker with an attack or a harmful spell while a duplicate remains, that creature rolls randomly to determine whether it targets the cloaker or one of the duplicates. a creature is unaffected by this magical effect if it can't see or if it relies on senses other than sight.
a duplicate has the cloaker's ac and uses its saving throws. if an attack hits a duplicate, or if a duplicate fails a saving throw against an effect tha

It looks like it is safe to ignore these as being duplicate attacks. Now lets see if we can come up with a way to model the attacks well. The plurals are most likely syntactic, and we can group them with the singular versions of themselves.

In [274]:
df_actions.Name = df_actions.Name.apply(lambda x : inf.singularize(x))
df_actions.groupby("Name").Type.count().head(20)

Name
acid breath           7
acid spray            1
animate chain         1
animate tree          1
antennae              1
battleaxe             1
beak                 13
beard                 1
bite                144
blinding breath       1
blinding spittle      1
blood drain           1
breath weapon        20
chain                 1
change shape         11
charm                 2
claw                 83
club                  4
cold breath           9
constrict             4
Name: Type, dtype: int64

In [276]:
df_actions["contains_attack"] = df_actions.Description.str.contains("attack")
df_actions["melee"] = df_actions.Description.str.contains("melee")
df_actions["ranged"] = df_actions.Description.str.contains("ranged")
df_actions.head()

/Users/rdunlop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/rdunlop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/rdunlop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,Monster Name,Name,Description,Type,melee,ranged,contains_attack
3,Aboleth,multiattack,the aboleth makes three tentacle attacks.,action,False,False,True
4,Aboleth,tentacle,"melee weapon attack: +9 to hit, reach 10 ft., ...",action,True,False,True
5,Aboleth,tail,"melee weapon attack: +9 to hit, reach 10 ft. o...",action,True,False,True
6,Aboleth,enslave,the aboleth targets one creature it can see wi...,action,False,False,False
13,Deva,multiattack,the deva makes two melee attacks.,action,True,False,True


In [292]:
df_attack_but_no_type = df_actions[(~(df_actions.melee) & ~(df_actions.ranged)) & df_actions.contains_attack & (df_actions.Name != 'multiattack')]
df_attack_but_no_type.Name.count()
df_attack_but_no_type.Description.head(30)

31      the solar releases its greatsword to hover mag...
59      the behir makes one bite attack against a medi...
89      the cloaker magically creates three illusory d...
96      the couatl magically polymorphs into a humanoi...
140     the quasit magically turns invisible until it ...
168     up to four chains the devil can see within 60 ...
190     the imp magically turns invisible until it att...
451     the dragon exhales gas in a 60-foot cone. each...
460     the dragon exhales gas in a 30-foot cone. each...
464     the dragon exhales gas in a 15-foot cone. each...
474     the dragon exhales gas in a 90-foot cone. each...
488     the dragon exhales gas in a 60-foot cone. each...
499     the dragon exhales gas in a 30-foot cone. each...
504     the dragon exhales gas in a 15-foot cone. each...
563     for 1 minute, the duergar magically increases ...
566     the duergar magically turns invisible until it...
625     one humanoid that the ghost can see within 5 f...
678     until 

There are 31 actions that do not contain either melee or ranged, but still have the word attack. These are all descriptions and not a simple direct attack. We will come back to these, but let's see if we can model the monsters based off of simple attacks. Let's see how many simple attacks each monster has.
# Simple attacks

In [302]:
df_actions['simple_attack'] = df_actions.melee | df_actions.ranged
df_actions.head()

/Users/rdunlop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Monster Name,Name,Description,Type,melee,ranged,contains_attack,simple_attack
3,Aboleth,multiattack,the aboleth makes three tentacle attacks.,action,False,False,True,False
4,Aboleth,tentacle,"melee weapon attack: +9 to hit, reach 10 ft., ...",action,True,False,True,True
5,Aboleth,tail,"melee weapon attack: +9 to hit, reach 10 ft. o...",action,True,False,True,True
6,Aboleth,enslave,the aboleth targets one creature it can see wi...,action,False,False,False,False
13,Deva,multiattack,the deva makes two melee attacks.,action,True,False,True,True


The total number of monsters will be useful

In [314]:
len(df_actions["Monster Name"].unique())

314

There are 314 monsters. How many contain at least one simple attack?

In [312]:
df_actions_simple = df_actions[df_actions.simple_attack]
len(df_actions_simple["Monster Name"].unique())

313

There are 313! We are only missing one monster. Who is it?

In [324]:
set(df_actions["Monster Name"].unique()).difference(set(df_actions_simple["Monster Name"].unique()))

{'Black Bear'}

So what's up with the black bear? Let's take a look.

In [340]:
print(df_actions.loc[df_actions['Monster Name'] == 'Black Bear'].Description.item())

the bear makes two attacks: one with its bite and one with its claws.


This makes it seem like there should be more to the bears attack...and there is! A quick check in the monster manual shows that the black bear contains a bite and a claw attack! Let's add these in.

In [348]:
df_black_bear = pd.DataFrame({"Monster Name":["Black Bear", "Black Bear"], 
                    "Name":["Bite", "Claws"],
                    "Description":["melee weapon attack: +3 to hit, reach 5 ft., one target. Hit: (1d6 + 2) piercing damage.","melee weapon attack: +3 to hit, reach 5 ft., one target. Hit: (2d4 + 2) slashing damage."],
                    "Type": ["action","action"],
                    "melee": [True,True],
                             "ranged": [False,False],
                             "contains_attack":[True,True],
                             "simple_attack":[True,True]})
df_actions = df_actions.append(df_black_bear)
df_actions_simple = df_actions[df_actions.simple_attack]
len(df_actions_simple["Monster Name"].unique())



314

Fixed! All monsters have at least one simple attack. It will be hard to quantify the non-simple attacks, so the next thing we should do is remove all non-simple attacks.